In [1]:
from tqdm.auto import tqdm
import torch 
from transformers import DistilBertModel
from transformers import DistilBertTokenizer
from datasets import Dataset
import pandas as pd
from transformers.optimization import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt 
import numpy as np
from sentence_transformers import SentenceTransformer
#Build up to SBERT model 

/opt/anaconda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/anaconda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/anaconda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/anaconda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
deviceNum = 1
device = torch.device("cuda:" + str(deviceNum) if torch.cuda.is_available() else "cpu")


In [3]:
def check_mem():
    torch.cuda.empty_cache()
    a = torch.cuda.memory_allocated(deviceNum)/1024/1024/1024
    r = torch.cuda.memory_reserved(deviceNum)/1024/1024/1024
    print("torch.cuda.memory_allocated: %fGB"%a)
    print("torch.cuda.memory_reserved: %fGB"%r)
    print("torch.cuda.memory_free: %fGB"%(r-a))
    print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(deviceNum)/1024/1024/1024))
check_mem()

torch.cuda.memory_allocated: 0.000000GB
torch.cuda.memory_reserved: 0.000000GB
torch.cuda.memory_free: 0.000000GB
torch.cuda.max_memory_reserved: 0.000000GB


In [4]:
#taken almost directly from: 
#https://www.pinecone.io/learn/train-sentence-transformers-softmax/

In [5]:
#clear up memory 

#torch.manual_seed(0)

In [6]:
#load data 
df = pd.read_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/enTrainData.csv", sep="\t")
df = df.loc[(df["url1_lang"] == "en") & (df["url2_lang"] == "en")]

#put ground truth values into a list 
df["ground_truth"] = df[["Geography", 'Entities', 'Time', 'Narrative', 'Overall', 'Style', 'Tone']].values.tolist()

#TODO: do we need to make data tensors or does this happen when we call set_format() later on
#df["ground_truth"] = df["ground_truth"].apply(torch.tensor)

#[torch.tensor(gtList) for gtList in df["ground_truth"]]

#get only the columns we need 
#TODO: do we need "pair_id"? 
leanDf = df[["ground_truth",  'text1', 'text2']].dropna()

print(leanDf.shape)
leanDf.head()

(1676, 3)


,ground_truth,text1,text2
0,"[4.0, 4.0, 1.0, 4.0, 4.0, 1.6666666666666667, ...","MARTINSBURG, W.Va. — A suspected drunken drive...","PORT-AU-PRINCE, Haiti — Haitian President Jove..."
1,"[4.0, 4.0, 1.0, 4.0, 3.6666666666666665, 1.666...",Share This On:\n\nPin 11 Shares\n\n(NEWS ROOM ...,BERLIN - A fire at a zoo in western Germany in...
2,"[1.0, 2.0, 1.0, 2.333333333333333, 2.333333333...",(Breitbart) – President Donald Trump welcomed ...,"PALM BEACH, United States — US President Donal..."
3,"[1.0, 2.333333333333333, 2.6666666666666665, 1...",Uber has sold its online food-ordering busines...,Rapid digitisation and growth in both online b...
4,"[1.0, 1.25, 1.0, 1.25, 1.25, 1.0, 1.0]",BENGALURU (Reuters) - India has approved its t...,BANGALORE: India plans to make a fresh attempt...


In [7]:
leanDf

,ground_truth,text1,text2
0,"[4.0, 4.0, 1.0, 4.0, 4.0, 1.6666666666666667, ...","MARTINSBURG, W.Va. — A suspected drunken drive...","PORT-AU-PRINCE, Haiti — Haitian President Jove..."
1,"[4.0, 4.0, 1.0, 4.0, 3.6666666666666665, 1.666...",Share This On:\n\nPin 11 Shares\n\n(NEWS ROOM ...,BERLIN - A fire at a zoo in western Germany in...
2,"[1.0, 2.0, 1.0, 2.333333333333333, 2.333333333...",(Breitbart) – President Donald Trump welcomed ...,"PALM BEACH, United States — US President Donal..."
3,"[1.0, 2.333333333333333, 2.6666666666666665, 1...",Uber has sold its online food-ordering busines...,Rapid digitisation and growth in both online b...
4,"[1.0, 1.25, 1.0, 1.25, 1.25, 1.0, 1.0]",BENGALURU (Reuters) - India has approved its t...,BANGALORE: India plans to make a fresh attempt...
...,...,...,...
2871,"[4.0, 3.0, 1.0, 2.0, 3.0, 1.0, 1.0]","Hello,\n\nWe are preparing to move our environ...",I'm running vCenter 6.7 w/ three host also run...
2872,"[1.0, 4.0, 3.0, 4.0, 4.0, 2.0, 1.0]",You can now consult with your doctors through ...,India is doing a commendable job in fighting t...
2873,"[1.0, 2.0, 1.0, 2.0, 1.0, 3.0, 1.0]",Jakraphanth Thomma on Saturday killed his comm...,Medics carry a stretcher towards a Thai shoppi...
2874,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",The bodies of the victims were found lying in ...,Johannesburg - Gauteng police on Saturday said...


In [8]:
import re
def getTopSentences(inStr):
    print(re.split("[.!]", inStr))
testStr = leanDf.loc[0,"text2"]
#print(testStr)
getTopSentences(testStr)

['PORT-AU-PRINCE, Haiti — Haitian President Jovenel Moïse broke with tradition on Wednesday and celebrated the country’s independence day in the capital for security reasons following months of political turmoil', '\n\nMoïse, whose government has been accused of corruption, denounced graft during his speech at the National Palace in Port-au-Prince and urged Haiti’s elite to work with the government and help create employment', '\n\n“We’re still extremely poor,” he said', ' “Those who continue to get rich find it normal that they do not pay taxes, find it normal that there can be no competition, find it normal that they set prices for consumers, especially when this consumer is the state itself', '”\n\nMoïse also apologized for the country’s ongoing power outages and renewed his 2016 campaign pledge to provide electricity 24 hours a day, saying it was harder to accomplish than he imagined', '\n\nThe speech that marked the 216th anniversary of the world’s first black republic was origina

In [9]:
#get data loaded in properly 
dataset = Dataset.from_pandas(leanDf)

In [10]:
tokenizer = DistilBertTokenizer.from_pretrained('bert-base-uncased')

all_cols = ["ground_truth"]

"""
for part in ['text1', 'text2']:
    dataset = dataset.map(
        lambda x: tokenizer(
            x[part], max_length=128, padding='max_length',
            truncation=True
        ), batched=True
    )
    
    for col in ['input_ids', 'attention_mask']:
        dataset = dataset.rename_column(
            col, part+'_'+col
        )
        all_cols.append(part+'_'+col)
print(all_cols)
"""

for part in ["text1", "text2"]: 
    #tokenizes each row of the dataset and gives us back tuple of lists 
    dataset=dataset.map(lambda x: tokenizer(x["text1"], max_length=100, padding='max_length', truncation=True))
    
    for col in ['input_ids', 'attention_mask']: 
        dataset = dataset.rename_column(
            col, part+'_'+col
        )
        all_cols.append(part+'_'+col)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


  0%|          | 0/1676 [00:00<?, ?ex/s]

  0%|          | 0/1676 [00:00<?, ?ex/s]

In [11]:
#we have the correct input format needed for the text data 
print(all_cols)

['ground_truth', 'text1_input_ids', 'text1_attention_mask', 'text2_input_ids', 'text2_attention_mask']


In [14]:
# convert dataset features to PyTorch tensors
dataset.set_format(type='torch', columns=all_cols)


In [15]:
check_mem()

torch.cuda.memory_allocated: 0.000000GB
torch.cuda.memory_reserved: 0.000000GB
torch.cuda.memory_free: 0.000000GB
torch.cuda.max_memory_reserved: 0.000000GB


In [16]:
length = len(dataset)
print("total rows: " + str(length))
trainProp = .75 

validLen = int(length * (1-trainProp))
trainLen = length - validLen
print("train rows: " + str(trainLen) + "\nvalidation rows: " + str(validLen) + "\nsum: " + str(trainLen + validLen))

train, valid = torch.utils.data.random_split(dataset, [trainLen, validLen])

total rows: 1676
train rows: 1257
validation rows: 419
sum: 1676


In [20]:
train

In [21]:

# initialize the dataloader
batch_size = 2
trainLoader = torch.utils.data.DataLoader(
    train, batch_size=batch_size, shuffle=True
)
validLoader = torch.utils.data.DataLoader(
    valid, batch_size=1, shuffle=True
)

In [22]:
def mean_pool(token_embeds, attention_mask):
    # reshape attention_mask to cover 768-dimension embeddings
    in_mask = attention_mask.unsqueeze(-1).expand(
        token_embeds.size()
    ).float()
    # perform mean-pooling but exclude padding tokens (specified by in_mask)
    pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(
        in_mask.sum(1), min=1e-9
    )
    return pool

In [23]:
check_mem()

torch.cuda.memory_allocated: 0.000000GB
torch.cuda.memory_reserved: 0.000000GB
torch.cuda.memory_free: 0.000000GB
torch.cuda.max_memory_reserved: 0.000000GB


In [24]:
# start from a pretrained bert-base-uncased model
# TODO: consider something already trained for sentance similarity? 
#del model 
model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(device)
#model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2').to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
check_mem()

torch.cuda.memory_allocated: 0.248280GB
torch.cuda.memory_reserved: 0.265625GB
torch.cuda.memory_free: 0.017345GB
torch.cuda.max_memory_reserved: 0.265625GB


In [26]:
#initialize the feed-forward NN first
#this will go from a pooled input to our 6 prediction labels 
ffnn = torch.nn.Linear(768*3, 7).to(device)

#TODO: double check on if reduction="mean" is the right move here...
#could cosine similarity also work..? I think that is between the two predicted vectors though.. 
loss_func = torch.nn.MSELoss(reduction="mean")

In [27]:
# we would initialize everything first
optim = torch.optim.Adam(model.parameters(), lr=2e-5)
# and setup a warmup for the first ~10% steps
total_steps = int(len(dataset) / batch_size)
warmup_steps = int(0.1 * total_steps)
scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=warmup_steps, num_training_steps=total_steps - warmup_steps)

In [28]:
dummyPreds = [[3, 1, 9], [8, 9, 10], [2, 4, 1], [10, 10, 9]]
dummyGT = [[2, 2, 6], [12, 5, 8], [3, 3, 3], [7, 8, 9]]

def getCorr(inPreds, inGT): 
    inPreds = np.array(inPreds)
    inGT = np.array(inGT)
    corrList = []
    for col in range(inPreds.shape[1]): 
        corr = np.corrcoef(inPreds[:,col], inGT[:,col])[0][1]
        corrList.append(corr)
    return corrList

print(getCorr(dummyPreds, dummyGT))

[0.7593948136311601, 0.9205664999205727, 0.8863416070394171]


In [33]:
check_mem()

torch.cuda.memory_allocated: 0.994450GB
torch.cuda.memory_reserved: 1.087891GB
torch.cuda.memory_free: 0.093441GB
torch.cuda.max_memory_reserved: 1.457031GB


In [34]:
trainDict = {}
# increase from 1 epoch if need be 
for epoch in range(1):
    
    model.train()  # make sure model is in training mode
    
    # initialize the dataloader loop with tqdm (tqdm == progress bar)
    loop = tqdm(trainLoader, leave=True)
    
    for i, batch in enumerate(loop):
        if i < 41: 
            # zero all gradients on each new step
            optim.zero_grad()

            # prepare batches and more all to the active device
            inputs_ids_a = batch['text1_input_ids'].to(device)
            inputs_ids_b = batch['text2_input_ids'].to(device)
            attention_a = batch['text1_attention_mask'].to(device)
            attention_b = batch['text2_attention_mask'].to(device)

            #ground truth labels need to go to device as well 
            label = batch['ground_truth'].to(device)

            # extract token embeddings from BERT
            u = model(
                inputs_ids_a, attention_mask=attention_a
            )[0]  # all token embeddings A

            v = model(
                inputs_ids_b, attention_mask=attention_b
            )[0]  # all token embeddings B

            # get the mean pooled vectors
            u = mean_pool(u, attention_a)
            v = mean_pool(v, attention_b)

            # build the |u-v| tensor
            uv = torch.sub(u, v)
            uv_abs = torch.abs(uv)

            # concatenate u, v, |u-v|
            x = torch.cat([u, v, uv_abs], dim=-1)

            # process concatenated tensor through FFNN
            x = ffnn(x)

            # calculate the 'MSE-loss' between predicted and true label
            loss = loss_func(x, label)

            
            #get validation loss
            model.eval()
            vLossList = []
            vCorrList = []
            vPred = []
            vGT = []
            if i % 20 == 0: 
                for vBatch in validLoader: 
                    # prepare batches and more all to the active device
                    vInputs_ids_a = vBatch['text1_input_ids'].to(device)
                    vInputs_ids_b = vBatch['text2_input_ids'].to(device)
                    vAttention_a = vBatch['text1_attention_mask'].to(device)
                    vAttention_b = vBatch['text2_attention_mask'].to(device)

                    #ground truth labels need to go to device as well 
                    vLabel = vBatch['ground_truth'].to(device)

                    # extract token embeddings from BERT
                    u = model(
                        vInputs_ids_a, attention_mask=vAttention_a
                    )[0]  # all token embeddings A

                    v = model(
                        vInputs_ids_b, attention_mask=vAttention_b
                    )[0]  # all token embeddings B

                    # get the mean pooled vectors
                    u = mean_pool(u, vAttention_a)
                    v = mean_pool(v, vAttention_b)

                    # build the |u-v| tensor
                    uv = torch.sub(u, v)
                    uv_abs = torch.abs(uv)

                    # concatenate u, v, |u-v|
                    valX = torch.cat([u, v, uv_abs], dim=-1)

                    # process concatenated tensor through FFNN
                    valX = ffnn(valX)

                    vLoss = loss_func(valX, vLabel)
                    vLossList.append(vLoss.item())
                    vPred.append(valX)
                    vGT.append(vLabel)
        
             #get training metrics 
            trainDict[i] = {"trainLoss": loss.item(), "testLoss":np.mean(vLossList)} 
            
            
            # using loss, calculate gradients and then optimize
            loss.backward()
            optim.step()

            # update learning rate scheduler
            scheduler.step()

            # update the TDQM progress bar
            loop.set_description(f'Epoch {epoch}')
            loop.set_postfix(loss=loss.item())

  0%|          | 0/629 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 1; 10.92 GiB total capacity; 7.16 GiB already allocated; 4.38 MiB free; 7.67 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [31]:
#check memory 
t = torch.cuda.mem_get_info()

used = torch.cuda.memory_allocated(device="cuda:4")
#proportion of free memory 
#print("used: " used / t[1])
print("used: " + str(t[0]/t[1]))

used: 0.4697167239540121


In [32]:
trainDf = pd.DataFrame.from_dict(trainDict, orient="index")
trainDf.head(3)

""


In [ ]:
fig, axs = plt.subplots(1,2, figsize=[14,4])
axs[0].plot(trainDf["trainLoss"])
axs[0].set_title("Train Loss", size=14)
axs[0].set_ylabel("MSE Loss",size=14)
axs[0].set_xlabel("batch",size=14)

axs[1].plot(trainDf.dropna()["testLoss"])
axs[1].set_title("Validation Loss",size=14)
axs[1].set_ylabel("MSE Loss",size=14)
axs[1].set_xlabel("batch",size=14)

In [ ]:
print(x)
print(label)
print()

In [ ]:
loss_func(x, label)

In [ ]:
plt.plot(trainDf["testLoss"].dropna())

In [ ]:
valX

In [ ]:
vLabel

In [ ]:
x

In [ ]:
label